In [17]:
import json
import cv2
import numpy as np
import tensorflow as tf

In [20]:
IMG_W = 448
IMG_H = 448
DEPTH = 3
NB_CELL = 7
NB_BOX = 2
NB_CLASS = 2
CLASSES = {
    'square': 0,
    'circle': 1
}



# label = [class]
def load(path):
    labs = json.load(open(path+'annotation.json'))
    N = len(labs)
    X = np.zeros((N, IMG_W, IMG_H, DEPTH), dtype=np.uint8)
    Y = np.zeros((N, NB_CELL, NB_CELL, 5+NB_CLASS)) #label 5 = confidance, x, y, w, h, class one hot
    Labels = []
    for i in range(len(labs)):
        # Normalise data
        X[i] =cv2.imread(path+f'img{i}.png')/255.
        for box in labs[i]['bboxes']:
            # one hot
            classe = [0]*NB_CLASS
            classe[CLASSES[box['classe']]] = 1
            x_center = box['x']
            y_center = box['y']
            w = box['w']
            h = box['h']
            x_index_cell, y_index_cell = int(x_center/IMG_W*NB_CELL), int(y_center/IMG_H*NB_CELL)
            try:
                Y[i, x_index_cell, y_index_cell] = 1, x_center, y_center, w, h, *classe
            except IndexError:
                print(i, x_index_cell, y_index_cell, x_center, y_center)
    return X, Y
            




In [21]:
path = 'data/'
X, Y = load(path=path)


In [27]:
Y[0][...,0]

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0.]])

In [22]:
from tensorflow.keras.layers import LeakyReLU

def createModelYOLOv1():
    # 24 layers Convo, leaky ReLu for all layers except final layer (lin activation)
    model = tf.keras.models.Sequential()
    #Convo_1
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2, 2), padding="valid", activation=LeakyReLU(alpha=0.1), input_shape=(448, 448, 3)))
    #MaxPool2D
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
    ########
    #Convo_2
    model.add(tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #MaxPool2D
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
    #######
    #Convo_3
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_4
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_5
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_6
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #MaxPool2D
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
    #######
    #Convo_7
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_8
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_9
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_10
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_11
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_12
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_13
    model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_14
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_15
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_16
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #MaxPool2D
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))
    #######
    #Convo_17
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_18
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_19
    model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_20
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_21
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #Convo_22
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(2, 2), padding=("same"), activation=LeakyReLU(alpha=0.1)))
    #######
    #Convo_23
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("same"), activation=LeakyReLU(alpha=0.1)))
    #Convo_24
    model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3,3), strides=(1, 1), padding=("same"), activation=LeakyReLU(alpha=0.1)))
    #######
    model.add(tf.keras.layers.Conv2D(filters=4096, kernel_size=(1,1), strides=(1, 1), padding=("valid"), activation=LeakyReLU(alpha=0.1)))
    #######
    model.add(tf.keras.layers.Conv2D(filters=5*NB_BOX+NB_CLASS, kernel_size=(1,1), strides=(1, 1), padding=("valid")))

    return model


In [23]:
model = createModelYOLOv1()